In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import shutil
import os 
import glob
import csv
import json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix as confusion_matrix_sklearn

from ketos.data_handling import selection_table as sl
import ketos.data_handling.database_interface as dbi
from ketos.data_handling.parsing import load_audio_representation
from ketos.data_handling.data_feeding import BatchGenerator
from ketos.neural_networks.resnet import ResNetInterface
from ketos.audio.audio_loader import AudioFrameLoader, AudioLoader, SelectionTableIterator
from ketos.audio.spectrogram import MagSpectrogram
from ketos.neural_networks.dev_utils.detection import batch_load_audio_file_data, filter_by_threshold, filter_by_label, merge_overlapping_detections
from ketos.data_handling.data_feeding import JointBatchGen

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print('done importing packages')

done importing packages


In [3]:
#annots = pd.read_excel(r'E:\final-baseline-detector\pearce-point-test\all_pp_annots.xlsx')
annots = pd.read_excel(r'E:\baseline-with-normalization-reduce-tonal\ulu2023\test-subset-annot-ulu2023.xlsx')

annotsf = annots.ffill()

audio_folder = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\audio'

for idex, row in annotsf.iterrows():
    shutil.copyfile(annotsf.loc[idex]['filename'], audio_folder + '\\' + annotsf.loc[idex]['filename'].split('\\')[-1])

print('done')

done


In [4]:
detections_csv = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\detections.csv'
audio_folder = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\audio'
temp_folder_pp = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\temp-audio'
model_name = r'E:\baseline-with-normalization-reduce-tonal\rs-model-3.kt'
spectro_file = r'E:\baseline-with-normalization-reduce-tonal\spec_config_100-1200Hz-0.032-hamm-normalized-reduce-tonal.json'

# Look at detections above this threshold
threshold = 0.5

# Step 0.5s each time (overlap of 50% for 1 sec duration)
step_size = 0.5

# Number of samples in batch
batch_size = 16

model = ResNetInterface.load(model_file=model_name, new_model_folder=temp_folder_pp)

audio_repr = load_audio_representation(path=spectro_file)

spec_config = audio_repr['spectrogram']

audio_loader = AudioFrameLoader(path=audio_folder, duration=spec_config['duration'],
                                    step=step_size, stop=False, representation=spec_config['type'],
                                    representation_params=spec_config, pad=False)
detections = pd.DataFrame()

batch_generator = batch_load_audio_file_data(loader=audio_loader, batch_size=batch_size)

for batch_data in batch_generator:
    # Run the model on the spectrogram data from the current batch
    batch_predictions = model.run_on_batch(batch_data['data'], return_raw_output=True)

    # Lets store our data in a dictionary
    raw_output = {'filename': batch_data['filename'], 'start': batch_data['start'], 'end': batch_data['end'],
                  'score': batch_predictions}

    batch_detections = filter_by_threshold(raw_output, threshold=threshold)

    detections = pd.concat([detections, batch_detections], ignore_index=True)

detections.to_csv(detections_csv, index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 9021/9021 [19:49<00:00,  7.58it/s]


In [5]:
grouped_detections_csv = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\pos_detections_ulu2023_audio-thresh05.csv'
audio_folder = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\audio'
raven_txt_name = r'E:\baseline-with-normalization-reduce-tonal\ulu2023\raven-formatted-detections-ulu2023-thresh05.txt'

In [6]:
# Filter the detections for only the positive results 
detections_filtered = filter_by_label(detections, labels=1).reset_index(drop=True)
print(detections_filtered)
detections_grp = merge_overlapping_detections(detections_filtered)
print(detections_grp)
detections_grp.to_csv(grouped_detections_csv, index=False)

results_table = pd.read_csv(grouped_detections_csv)

cols = ['filename']
results_table.loc[:,cols] = results_table.loc[:,cols].ffill()
results_table['Selection'] = results_table.index +1
results_table['View'] = 'Spectrogram 1'
results_table['Channel'] = 1
results_table['Begin Path'] = audio_folder + '\\' + results_table.filename
results_table['File Offset (s)'] = results_table.start
results_table = results_table.rename(columns={"start": "Begin Time (s)", "end": "End Time (s)", "filename": "Begin File"})
results_table['Begin File'] = results_table['Begin File']
results_table['Low Freq (Hz)'] = 100
results_table['High Freq (Hz)'] = 1200

results_table.to_csv(raven_txt_name, index=False, sep='\t')

                   filename  start    end  label     score
0     7266.230518115003.wav  139.5  140.5      1  0.918850
1     7266.230518115003.wav  140.0  141.0      1  0.993072
2     7266.230518115003.wav  211.0  212.0      1  0.552933
3     7266.230518115503.wav  199.0  200.0      1  0.999987
4     7266.230518115503.wav  200.0  201.0      1  0.999965
...                     ...    ...    ...    ...       ...
9903  7266.230521101015.wav  292.5  293.5      1  0.999991
9904  7266.230521101015.wav  293.0  294.0      1  1.000000
9905  7266.230521101015.wav  293.5  294.5      1  1.000000
9906  7266.230521101015.wav  294.0  295.0      1  0.999025
9907  7266.230521101015.wav  294.5  295.5      1  0.992711

[9908 rows x 5 columns]
                   filename  start    end  label     score
0     7266.230518115003.wav  139.5  141.0      1  0.955961
1     7266.230518115003.wav  211.0  212.0      1  0.552933
2     7266.230518115503.wav  199.0  203.0      1  0.999621
3     7266.230518120003.wav   3